# 引导区

In [1]:
import json
import requests
import pandas as pd
from pprint import pprint
from pandas.io.json import json_normalize

# 页面解析函数

## 页面解析函数

### 查询
0. 备注
    * 数据全部以DF格式返回
1. query_ticket
    * 输入日期、起点站代码、目的站代码、车票类型(成人/?)，返回可以购买的车次的车票数据（多行）
    * 未解决：解析返回的复杂字符串（数据包含于其内）
        * result里的每一行车次数据的第一个复杂字符串是火车的信息，在后面提交订单的时候就是提交这段字符。这段字符是随机生成的，过几秒就会失效
2. query_price
    * 输入列车代码、起点站代码、目的站代码、`座位类型?`、日期，返回票价数据
3. query_station
    * 输入列车代码、起点站代码、目的站代码、日期，返回列车全程站点数据

In [15]:
#输入日期、起点站代码、目的站代码、车票类型(成人/?)，返回可以购买的车次的车票数据（多行）
def query_ticket(train_date, from_station, to_station, ticket_type):
    url='https://kyfw.12306.cn/otn/leftTicket/query'
    payload = {'leftTicketDTO.train_date': train_date, #日期
           'leftTicketDTO.from_station': from_station, #起点站
           'leftTicketDTO.to_station': to_station, #目的站
           'purpose_codes': ticket_type} #车票类型
    r = requests.get(url, params=payload)
    return r.json()
#     return r.text

#输入日期、起点站代码、目的站代码、车票类型，返回相关数据
def query_price(train_no, from_station_no, to_station_no, seat_types, train_date):
    url='https://kyfw.12306.cn/otn/leftTicket/queryTicketPrice'
    payload = {'train_no': train_no, #列车代码
           'from_station_no': from_station_no, #起点站序号
           'to_station_no': to_station_no, #目的站序号
           'seat_types': seat_types, #座位类型
           'train_date': train_date} #日期
    r = requests.get(url, params=payload)
    return r.json()

#输入列车代码、起点站代码、目的站代码、日期，返回列车全程站点数据
def query_station(train_no, from_station, to_station, date):
    url='https://kyfw.12306.cn/otn/czxx/queryByTrainNo'
    payload = {'train_no': train_no, #列车代码
           'from_station_telecode': from_station, #起点站序号
           'to_station_telecode': to_station, #终点站序号
           'depart_date': date} #日期
    r = requests.get(url, params=payload)
    return r.json()


## 数据变换函数

### 数据变换
1. price_json_df
2. trans_ticket

In [4]:
#或许还是存为DF格式比较好？
def price_json_df(json_data):
    
    dict_data={}
    dict_data['商务/特等']=json_data['data'].get('A9')
    dict_data['一等座']=json_data['data'].get('M')
    dict_data['二等座']=json_data['data'].get('O')
    dict_data['高级软卧']=json_data['data'].get('A6')
    dict_data['软卧']=json_data['data'].get("A4")
    dict_data['动卧']=json_data['data'].get('F')
    dict_data['硬卧']=json_data['data'].get('A3')
    dict_data['软座']=json_data['data'].get('A2')
    dict_data['硬座']=json_data['data'].get('A1')
    dict_data['无座']=json_data['data'].get('WZ')
#     dict_data['其他']=json_data['data'][]
    
    return dict_data

#将query_ticket查询得到的数据根据映射关系写入DF
def trans_ticket(ticket_list):
    df_empty = pd.DataFrame(columns=['起点站', '列车编号', '终点站', '出发站', '到达站', '出发时间', '到达时间',
                                     '历时', '起点站发车日期', '是否可以预订', '备注', 'seat_types', '软卧',
                                     '无座', '商务座/特等座', '一等座', '二等座'])
    for ticket in ticket_list:
        ticket_split = ticket.split('|')
        print(ticket_split[3], ticket_split[30])
        pprint(ticket_split)
        df_empty.loc[ticket_split[3],['备注', '列车编号', '起点站', '终点站', '出发站',
                                      '到达站', '出发时间', '到达时间',
                                      '历时', '是否可以预订', '起点站发车日期', '软卧', '无座',
                                      '二等座', '一等座', '商务座/特等座', 'seat_types']] = [ticket_split[1], ticket_split[2], 
                                                      ticket_split[4],
                                                      ticket_split[5], ticket_split[6], ticket_split[7],
                                                      ticket_split[8], ticket_split[9], ticket_split[10], 
                                                      ticket_split[11], ticket_split[13], ticket_split[23],
                                                      ticket_split[26], ticket_split[30], ticket_split[31],
                                                      ticket_split[32], ticket_split[35]]

    return df_empty


# 跳转函数

# 主函数

In [16]:
data = query_ticket('2018-07-23', 'BJP', 'SHH', 'ADULT')['data']['result'] #北京-上海
# data = query_ticket('2018-07-23', 'BJP', 'SHH', 'ADULT')
data
# data = query_ticket('2018-07-07', 'CQW', 'GZQ', 'ADULT')['data']['result'] #重庆-广州
# data = query_ticket('2018-07-07', 'HBB', 'GZQ', 'ADULT')['data']['result'] #哈尔滨-广州
# data = query_ticket('2018-07-07', 'TJP', 'GZQ', 'ADULT')['data']['result'] #天津-广州
# pprint(data)
# df_empty=trans_ticket(data)
# df_empty
# js=query_price('240000T10911', '01', '09', '14163', '2018-07-09')
# js
# price_json(js)
# query_station('240000D9010F', 'BXP', 'IZQ', '2018-07-07')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
js=query_price('240000T10911', '01', '09', '14163', '2018-07-09')
js

{'validateMessagesShowId': '_validatorMessage',
 'status': True,
 'httpstatus': 200,
 'data': {'3': '3045',
  'A1': '¥177.5',
  '1': '1775',
  'A4': '¥476.5',
  'A3': '¥304.5',
  'A6': '¥879.5',
  '6': '8795',
  '4': '4765',
  'OT': [],
  'WZ': '¥177.5',
  'train_no': '240000T10911'},
 'messages': [],
 'validateMessages': {}}